In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras

In [4]:
from keras.preprocessing.text import Tokenizer

In [5]:
from keras.preprocessing.sequence import pad_sequences

In [6]:
from tensorflow.keras import Sequential

In [7]:
from tensorflow.keras.layers import GRU, Embedding

In [8]:
from tensorflow.keras.layers import Dense

In [9]:
from tensorflow.keras.activations import sigmoid

In [10]:
from keras.optimizers import Adam

In [11]:
dataset = pd.read_csv('hepsiburada.csv')

In [12]:
dataset

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
243492,1,fiyatına göre güzel
243493,1,Ürün kullanışlı iş görüyor fazlasıyla eşime al...
243494,1,"Hızlı Kargo, güzel ürün"
243495,1,telefon başarılı hızlı bir cihaz sadece beyaz...


In [13]:
target = dataset['Rating'].values.tolist() #etiketler yani sütunlar liste haline çevrildi.
data = dataset['Review'].values.tolist()  #yorumlar yani satırlar liste haline geldi 

In [14]:
cutoff = int(len(data) * 0.80)  #verilerin %80'i eğitim için kullanıldı.
x_train, x_test = data[:cutoff], data[cutoff:] #ayırdığımız verileri ikiye ayırdık.
y_train, y_test = target[:cutoff], target[cutoff:] #aynı zamanda etiketleride ikiye ayırdık.

In [15]:
x_train[500] #listedeki 500.sıradaki yorum

'ürün sipariş verdim 2 gün içinde elime ulaştı her zaman ki gibi kullanışlı bi ürün daha once de bu mouse dan almıştım.bu yüzden tereddütsüz aldım . alacak olanlara öneririm'

In [16]:
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [17]:
y_train[800] #etiketteki yorumu gösterir.

1

In [18]:
num_words = 10000 #kelime haznemizdeki kelime sayısı yani datasetin içerisinde geçen en sık 10000 kelimeyi almış oluyoruz.
tokenizer = Tokenizer(num_words=num_words)

In [19]:
tokenizer.fit_on_texts(data) #tokenleştirme işlemi


In [20]:
tokenizer.word_index #datasetinde en çok kullanılan kelimeler çoktan aza göre sıralanıcak.

{'çok': 1,
 'bir': 2,
 've': 3,
 'ürün': 4,
 'bu': 5,
 'iyi': 6,
 'güzel': 7,
 'için': 8,
 'tavsiye': 9,
 'ederim': 10,
 'daha': 11,
 'ama': 12,
 'da': 13,
 'gayet': 14,
 'hızlı': 15,
 'teşekkürler': 16,
 'aldım': 17,
 'de': 18,
 'ürünü': 19,
 'gibi': 20,
 'yok': 21,
 'uygun': 22,
 'olarak': 23,
 'kaliteli': 24,
 'en': 25,
 '2': 26,
 'kargo': 27,
 'fiyat': 28,
 'elime': 29,
 'kadar': 30,
 'ile': 31,
 'göre': 32,
 'geldi': 33,
 'var': 34,
 'hepsiburada': 35,
 'ben': 36,
 'gerçekten': 37,
 '1': 38,
 'fiyata': 39,
 'gün': 40,
 'sonra': 41,
 'cok': 42,
 'kesinlikle': 43,
 'telefon': 44,
 'biraz': 45,
 'hiç': 46,
 'ulaştı': 47,
 'memnun': 48,
 'hem': 49,
 'değil': 50,
 'kullanışlı': 51,
 '3': 52,
 'mükemmel': 53,
 'oldu': 54,
 'kullanıyorum': 55,
 'önce': 56,
 'sipariş': 57,
 'tek': 58,
 'her': 59,
 'bence': 60,
 'harika': 61,
 'kalitesi': 62,
 'bi': 63,
 'ayrıca': 64,
 '5': 65,
 'teşekkür': 66,
 'fiyatı': 67,
 'olması': 68,
 'ne': 69,
 'herkese': 70,
 'bile': 71,
 'uzun': 72,
 'süper': 73,

In [21]:
x_train_tokens = tokenizer.texts_to_sequences(x_train) #eğitim setini tokenleştirme işlemi

In [22]:
x_train[800] #800.yoruma bakalım

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [23]:
print(x_train_tokens[800]) #800.yorumun token hali 

[19, 341, 52, 194, 30, 54, 7992, 55, 209, 603, 7887, 3, 36, 114, 164, 479, 85, 1, 1682, 782, 30, 2, 380, 326, 3, 1, 81, 132, 562, 9, 10]


In [24]:
#yani x_train içindeki kelimeler yukarıda görüldü gibi her kelimeye karşılık gelen integer sayılardan oluşturuldu.
#tabiki cümlemiz içinde başta seçtğimiz 10000 kelime arasında olan kelimelerden yoksa o kelimeyi yok sayacaktır en sık 
#kullanılan 10000 kelimeyi içerdiğinde atadığı integer sayıları görmüş oluyoruz.

In [25]:
x_test_tokens = tokenizer.texts_to_sequences(x_test) #test setimizi tokenleştiriyoruz.

In [26]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens] #her yorumda kaç token olduğunu hesaplayarak listeye yazar
num_tokens = np.array(num_tokens) #liste üzerinde işlem yapmayı kolaylaştırma açısından diziye dönüştürdüm.

In [27]:
np.mean(num_tokens) #yorumlarda ortalama kaç token olduğunu görmek için

20.744703220162876

In [28]:
np.max(num_tokens) #en fazla token bulunduran yorum kaç token sayısı bulunduruyor bunu gösterir.

295

In [29]:
np.argmax(num_tokens) #argmax ile listedeki en büyük elemanın indexini buluruz.
#en uzun yorumun indexi:

21941

In [30]:
#datasetimizdeki en uzun yoruma bakmak için:
x_train[21941]

'Özellikle bu kısma yazıyorum iyice okuyunuz,cihazın hızı çok iyi.Isınma normal boyutlarda.Kamerası elinizi sabit tutarsanız ve gündüz çok net gece ise çok net çekmez.Görüntülü konuşma özelliği yok ancak uygulama ile olur,anten çekimi gayet iyi.Parmak izi okuyucusu gayet hızlı bazen tuşu silmenizi istiyor ve okuyamıyor kirden vs...Ön kamera da tatmin edici,çekim esnasında ekran beyaz ışık vererek flaş görevi görür.Batarya bana 1 hafta gidiyor sık kullanımda ise 2-3 gün gidiyor.Hızlı şarj 1 saatte doluyor , şekilleri ele oturuyor ve şık bir görüntü var.Telefonu aldığım gün gittim ve ilk girdiğim yerden ekran koruyucu ve kılıf buldum.Kulaklık sesi çok net ve yüksek ancak kendi hoparlörü biraz zayıf sesi.Ekrana bakarken açık kalma özelliği yok.Diğer akıllı cihaz özellikleri %90 ı bu cihazda mevcut.Güç tasarrufu 2 ayrı modu var ve çok başarılı çalışıyor.4gb ram var genelde yarısı boş kalıyor.Bir de yeni cihazların çoğu titreşimi az ve sesi de az çıkıyor.Bu cihaz da içine dahil...Bu cihazla

In [31]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens) #Padding işlemi: yorumların boyutlarını belirlemek için
max_tokens = int(max_tokens)                              #ortalama bir boyut belirleme işlemidir 
max_tokens        

59

In [32]:
#yorumların kaç tanesi 59 token bulundurduğuna bakalım.
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9597982726686571

In [33]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens) #padding
#eğitim setindeki her yorumu eşit boyuta getirdik yani her yorum 59 uzunluğunda olucak

In [34]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens) #padding
#aynı işlemi test setimiz içinde gerçekleştirdim. Artık tüm yorumların uzunluğu 59 olmuş oldu.

In [35]:
x_train_pad.shape #padding eklenmiş mi diye test ediyoruz.

(194797, 59)

In [36]:
x_test_pad.shape

(48700, 59)

In [37]:
np.array(x_train_tokens[800]) #800.yorumun tokenlerine bakıyoruz. Burada token sayısının 59'dan az olduğunu görüyoruz

array([  19,  341,   52,  194,   30,   54, 7992,   55,  209,  603, 7887,
          3,   36,  114,  164,  479,   85,    1, 1682,  782,   30,    2,
        380,  326,    3,    1,   81,  132,  562,    9,   10])

In [38]:
x_train_pad[800] #800.yorumun 59'a tamamlanıp tamamlanmadığını kontrol ediyoruz. Başına sıfır eklenerek 59'a tamamlanmış

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,   19,  341,   52,  194,   30,
         54, 7992,   55,  209,  603, 7887,    3,   36,  114,  164,  479,
         85,    1, 1682,  782,   30,    2,  380,  326,    3,    1,   81,
        132,  562,    9,   10])

In [39]:
#Tokenleri verip stringi görebilmemizi sağlaması için bir fonksiyon yazıcam böylece bu fonksiyonu çağırdığımızda elimizdeki 
#tokenleri yoruma çevirmiş olacak.

In [40]:
idx = tokenizer.word_index                        #sözlüğü tersine çevirme
inverse_map = dict(zip(idx.values(), idx.keys())) #kelimeler = values, sayılar = key, 

In [41]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [42]:
#fonksiyonu test etme 
x_train[800] #800.yorum tokenleştirmeden önceki hali

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [43]:
tokens_to_string(x_train_tokens[800]) #burada tokenleştrime yapılmış yani her bir kelimeye integer değer atandıktan sonra 
                                      #tekrardan string haline gelmiş hali

'ürünü alalı 3 hafta kadar oldu aralıksız kullanıyorum sürekli açık durur ve ben günde yaklaşık 12 saat çok kullanırım şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim'

In [44]:
# En sık kullanılan kelimeleri 10000 ile sınırlandırmıştır bazı kelimelerin çıkarılmış olması bu 10000 kelimeden biri
# olmamasından kaynaklı.

In [45]:
model = Sequential() #Ardışık bir model oluşturduk.

In [46]:
embedding_size = 50 #embedding matrisinin büyüklüğünü belirleme

In [47]:
#Embedding layers
model.add(Embedding(input_dim=num_words,        #Matrisimiz kelime sayısı ve Embedding sayısı büyüklüğünde olacaktır. ilk önce kelime sayımızı atadık.
                    output_dim=embedding_size,  #Kelime vektörlerinin uzunluğunu atadık. (10000 x 50 uzunluğunda matris)
                    input_length=max_tokens,    #Gelen inputun boyutu
                    name='embedding_layer'))    #bu matristeki weightleri çağırmak için isim atadık.

In [48]:
# 3 katlı Yinelenen Sinir Ağı GRU layer
model.add(GRU(units=16, return_sequences=True)) #unit:GRU layerın nöron sayısı, 16 tane output verir, return olarak outputun tamamı döner
model.add(GRU(units=8, return_sequences=True))  #2. GRU layer
model.add(GRU(units=4))                         #3. GRU layer son layer lduğu içinde return sequences:false olmalı
model.add(Dense(1, activation='sigmoid'))       #3 katlı GRU da oluşan son output tek bir nörondur.

In [49]:
#Modelin eğitim optimizasyonu
optimizer = Adam(learning_rate=0.001) 

In [50]:
optimizer = 'adam'

In [51]:
#Model belirleme
model.compile(loss='binary_crossentropy',  #2 sınıf olduğu için binary crossentropy
              optimizer='adam',            
              metrics=['accuracy'])        #Modelin başarı oranı

In [52]:
model.summary()  #Modelin özeti

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding  (None, 59, 50)            500000    
 )                                                               
                                                                 
 gru (GRU)                   (None, 59, 16)            3264      
                                                                 
 gru_1 (GRU)                 (None, 59, 8)             624       
                                                                 
 gru_2 (GRU)                 (None, 4)                 168       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 504061 (1.92 MB)
Trainable params: 504061 (1.92 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [53]:
#NumPy dizisini TensorFlow Tensor objesine dönüştürme.
x = tf.convert_to_tensor(x_train_pad) 
y = tf.convert_to_tensor(y_train)

In [54]:
#Datasetindeki yorumları ve etikerleri vererek eğitim yapmak
#Bu fonksiyonu kullanabilmek için yukarıdaki satırda Tensor objesine çevirdik. 
#x:Eğitim setindeki yorumlar
#y:Eğitim setindeki yorumların etiketleri
model.fit(x,y, epochs=5, batch_size=256) #epoch:Eğitim sayısı

Epoch 1/5


761/761 [==============================] - 73s 84ms/step - loss: 0.1681 - accuracy: 0.9545
Epoch 2/5
761/761 [==============================] - 68s 89ms/step - loss: 0.0890 - accuracy: 0.9712
Epoch 3/5
761/761 [==============================] - 71s 93ms/step - loss: 0.0703 - accuracy: 0.9780
Epoch 4/5
761/761 [==============================] - 70s 92ms/step - loss: 0.0566 - accuracy: 0.9831
Epoch 5/5
761/761 [==============================] - 72s 94ms/step - loss: 0.0453 - accuracy: 0.9874


In [55]:
#Verileri numpy array dizisine dönüştürme
x_test_pad = np.array(x_test_pad) 
y_test = np.array(y_test)

In [56]:
#Modeli test etme
result = model.evaluate(x_test_pad, y_test) #Test setimizin yorumları, etiketleri

1522/1522 [==============================] - 29s 18ms/step - loss: 0.1510 - accuracy: 0.9557


In [57]:
result[1] #Test setimizin sonucu 

0.9556878805160522

In [58]:
#Bu sonuca göre modelimmiz test setinin %95 ini doğru bildiğini görmüş oluyoruz.

In [59]:
x_model_train = model.predict(x=x_train_pad[0:10000]) # Modelin eğitim setindeki yorumlar hakkındaki tahmini 
x_model_train

313/313 [==============================] - 8s 18ms/step


array([[0.966403  ],
       [0.996515  ],
       [0.99729437],
       ...,
       [0.99726206],
       [0.99690646],
       [0.9973081 ]], dtype=float32)

In [60]:
x_train_negative = np.array([1.0 if n>0.5 else 0.0 for n in x_model_train]) # Modelin eğitim setinde yapmış olduğu tahminleri 0 ve 1 olarak ikiye ayırdık
zero_indices = np.where(x_train_negative == 0.0)[0]                         # Modelin sadece olumsuz olarak tahmin ettiği yorumların indexlerini ve 
zero_values = x_model_train[zero_indices]                                   # modelin tahmin değerini gösterir. 
print("****************Sonucu 0 olan değerlerin indexleri:****************",'\n', zero_indices)
print("****************Sonucu 0 olan değerlerin değerleri:****************",'\n', zero_values)

****************Sonucu 0 olan değerlerin indexleri:**************** 
 [  20  194  225  527  528  785  891  905  914  999 1028 1029 1032 1034
 1476 1510 1516 1518 1527 1533 1680 1791 1995 2062 2065 2066 2069 2072
 2073 2079 2087 2101 2104 2139 2206 2213 2221 2223 2228 2232 2235 2248
 2261 2269 2271 2282 2291 2292 2293 2305 2313 2316 2321 2325 2331 2332
 2333 2343 2348 2349 2351 2355 2358 2359 2361 2371 2378 2379 2382 2383
 2411 2413 2433 2476 2540 2653 2655 2665 2669 2712 2718 2736 2819 2866
 2874 2877 2881 2948 2971 3000 3385 3425 3481 3497 3745 3756 3761 3762
 3778 3800 3856 3915 3964 3981 3992 4001 4003 4004 4009 4019 4123 4144
 4166 4203 4207 4208 4237 4410 4419 4435 4439 4442 4444 4445 4452 4484
 4493 4494 4502 4510 4520 4533 4538 4547 4583 4613 4614 4618 4629 4645
 4926 4954 4983 5001 5275 5398 5417 5423 5426 5433 5440 5472 5492 5497
 5530 5532 5540 5546 5549 5555 5557 5591 5606 5647 5651 5656 5676 5677
 5684 5686 5691 5704 5719 5726 5736 5763 5846 5851 5858 5922 5924 5925
 5934 5

In [61]:
x_model_train[2293]  # Yukarıdaki olumsuz yorumlardan birinin indexini yazarak modelin tahmin değerini verir.

array([0.02056496], dtype=float32)

In [62]:
x_train[2293]  # Bu olumsuz yorumun string hali

'Dış görünüş ve tutuş fena değil. Tuşlara macro ayarlanmıyor. Birde mouse 0.5cm kaldırdığınızda bile tepki veriyor. Bu yüzden hiç beğenmedim zemine temas etmediği halde mouse iconu hareket ediyor. Online oyunculara hitap etmeyen vasat bir ürün.'

In [63]:
x_model_train[49] # Yukarıdaki yorumlardan hariç girdiğimiz bir indexin sonucu 1'e yakın olduğu için modelimiz olumlu bir yorum oalrak tahmin etmiş.

array([0.99080867], dtype=float32)

In [64]:
x_train[49]  # Bu yorumun string halini görerek yorumun olumlu mu olumsuz mu olduğunu kendimizde değerlendirebiliriz

'Ürün çok kaliteli vereceğiniz paraya değecektir.'

In [65]:
# Modelin tahmin değeri 1'e yakınsa olumlu bir yorum, 0'a yakınsa olumsuz bir yorum şeklinde eğitilmiştir.
# Buradaki sonuclaraa bakılarak modelimiz olumlu ve olumsuz şeklinde iyi ayırt edebilmeyi başarmıştır.

In [66]:
x_model_test = model.predict(x=x_test_pad[0:10000]) # Modelin test setindeki yorumlar hakkındaki tahmini 
x_model_test

313/313 [==============================] - 5s 17ms/step


array([[0.9967153 ],
       [0.9973649 ],
       [0.9964724 ],
       ...,
       [0.9972554 ],
       [0.99696034],
       [0.9679889 ]], dtype=float32)

In [67]:
x_test_negative = np.array([1.0 if n>0.5 else 0.0 for n in x_model_test])
zero_indices_test = np.where(x_test_negative == 0.0)[0]  
zero_values_test = x_model_test[zero_indices_test]
print("****************Sonucu 0 olan değerlerin indexleri:****************",'\n', zero_indices_test)
print("****************Sonucu 0 olan değerlerin değerleri:****************",'\n', zero_values_test)

****************Sonucu 0 olan değerlerin indexleri:**************** 
 [  34   57   67   89   93  128  144  148  222  225  236  251  253  259
  277  284  290  293  308  319  339  353  429  434  440  441  445  465
  468  473  475  478  499  517  527  569  601  611  616  638  640  651
  662  668  699  717  733  740  762  794  800  852  859  876  940  944
  949  951  982 1002 1010 1012 1013 1018 1025 1036 1038 1040 1041 1050
 1053 1058 1061 1064 1066 1090 1098 1101 1114 1115 1122 1149 1153 1163
 1180 1197 1214 1218 1228 1259 1261 1269 1281 1296 1308 1315 1316 1330
 1352 1354 1383 1408 1439 1459 1462 1464 1468 1510 1539 1554 1559 1582
 1584 1600 1668 1698 1711 1733 1749 1764 1807 1873 1874 1877 1882 1883
 1888 1906 1912 1917 1928 2052 2055 2059 2063 2083 2135 2180 2204 2303
 2329 2453 2488 2516 2527 2534 2549 2552 2560 2569 2574 2596 2600 2605
 2606 2619 2646 2649 2657 2670 2671 2679 2711 2721 2726 2759 2847 2933
 2950 2975 2985 2998 3003 3046 3082 3189 3210 3238 3277 3293 3295 3330
 3427 3

In [68]:
x_model_test[180] # 180.yorumun model tahmini

array([0.9972023], dtype=float32)

In [69]:
x_test[180] # 180.yorum

'10 Yıldır kullanıyorum.Memnunum.'

In [70]:
x_model_test[951]

array([0.01506572], dtype=float32)

In [71]:
x_test[951]  

'Malesef kedim beğenmedi koklayıp koklayıp geri gidiyor inat ettik yesin diye ama 2 gündür bir lokma yemeden aç bekliyor'

In [72]:
# Modelin yaptığı hatalara bakmak
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]      #Modeli ilk 1000 üzerinde çalıştırmak bu yorumlarda yanlış tahmin ettiği yorumları bulmak için bir matris oluşturduk.

32/32 [==============================] - 1s 23ms/step


In [73]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred]) # Modelin tahminlerini 1 veya 0 olarak atama işlemi.

In [74]:
cls_true = np.array(y_test[0:1000]) 

In [75]:
incorrect = np.where(cls_pred != cls_true) # Karşılaştırma işlemi
incorrect = incorrect[0]

In [76]:
len(incorrect) # Kaç tane yorumun yanlış seçildiğini gösterir.

53

In [77]:
idx = incorrect[9] # Yanlış bilinen yorumun indexi
idx

149

In [78]:
text = x_test[idx] # İndexteki yanlış yorumu görme
text

'Ürünü alalı çok olmadı, tarifine göre ektim bakalım. 4-5 gün oldu bazı tohumların uçları görünmeye başladı. Plastik kap beni de şaşırttı aslında, resimde ki gibi bir yükselti ya da karton kılıf bekliyordum.   Çıksınlar bakalım kedim umarım sever. Ama arkadaşların dediği gibi çim yerken yerinden çıkıyorsa kötü, benim kedim 4 aylık daha ön dişleri çok keskin değil..'

In [79]:
y_pred[idx] # Modelin idx index deki yoruma tahmini

0.5977849

In [80]:
#Model yorumun olumlu olduğunu düşünüyor fakat yorum olumsuz bu yüzden model bu yorumu yanlış tahmin etmiş.

In [81]:
cls_true[idx] # Bu yorumun olumsuz olduğunu gösterir.

0

In [82]:
#Kendi yorumlarımızı yazarak da modelin yorumlar hakkındaki olumlu ya da olumsuz tahminlerini görebiliriz.

In [83]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
text11 = "fiyat olarak fena değil ama performans olarak kötü" 
text12 = "bölye bir ürün görmedim, çöp"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10,text11,text12]

In [84]:
tokens = tokenizer.texts_to_sequences(texts) 

In [85]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

(12, 59)

In [86]:
model.predict(tokens_pad)

1/1 [==============================] - 0s 35ms/step


array([[0.9972661 ],
       [0.9949449 ],
       [0.07316323],
       [0.99662715],
       [0.13908598],
       [0.09518111],
       [0.9942308 ],
       [0.02050664],
       [0.99604714],
       [0.36644265],
       [0.0799331 ],
       [0.04755048]], dtype=float32)